# _Redes neuronales artificiales_

### Importaciones

In [44]:
import random
import numpy as np
from math import e 

### Visión general de la implemetación
Se definen las clases **Neuron** y **ANN** las cuales representan de forma general a las neuronas y a la red neuronal a entrenear respectivamente. <br>
Dados estos modelos, un objeto de tipo ANN se asocia a varios objetos de tipo Neuron. <br>
Dicha asociación se da a través de dos listas de neuronas que contiene la clase ANN, una lista representando a la capa oculta y la restante a la capa de salida. <br>
Adicionalmente el objeto ANN recibe como parámetro el número de neuronas que debe implementar en cada capa. <br>
Dada esta arquitectura, solo es posible construir redes con una única capa oculta pero con capas de distinto tamaño. 

### Clase Neuron
***
#### Métodos
- **init(n_inputs)**: Inicializa n_inputs pesos de la neurona con valores entre -0.5 y 0.5 de forma aleatoria.
- **compute_input(inputs)**: Calcula la combinación lineal entre los pesos y los valores de las inputs.
- **apply_sigmoid(computed_value)**: Aplica la función de sigmoide a computed_value y almacena el valor de salida.
- **calculate_output_neuron_error(expected)**: Aplica la formula de error para una neurona de la capa de salida y almacena el resultado. 
- **calculate_hidden_neuron_error(output_layer, index)**: Aplica la formula de error para una neurona de la capa de oculta y almacena el resultado.
- **update_weights(rate, inputs)**: Actualiza todos los pesos de la neurona incluyendo el peso $w_{0}$ para el cual se asume una entrada de valor 1.

#### Observaciones
- Se asume que el valor $w_{0}$ es el último en la lista de pesos para mayor facilidad a la hora de implemetación.


In [33]:
class Neuron:
    def __init__(self, n_inputs):
        self.weights = [round(random.uniform(-0.5,0.5),5) for i in range(n_inputs)]
        self.output = Null
        self.error = Null

    def compute_input(self, inputs):
        result = self.weights[-1]
        for i in range(len(self.weights)-1):
            result += self.weights[i] * inputs[i]
        return result
    
    def apply_sigmoid(self, computed_value):
        self.output = 1 / (1 + e**(-computed_value)) 
        return self.output
    
    def calculate_output_neuron_error(self, expected):
        self.error = self.output * (1.0 - self.output) * (expected - self.output) 
        return  self.error
    
    def calculate_hidden_neuron_error(self, output_layer, index):
        self.error = 0.00000
        for neuron in output_layer:
            self.error += neuron.error * neuron.weights[index]
        self.error *= self.output * (1.0 - self.output)
        return self.error
    
    def update_weights(self, rate, inputs):
        for i in range(len(self.inputs)):
            self.weights[i] += rate * self.error * inputs[i]
        self.weights[-1] += rate * self.error  
    

### Clase ANN
***
#### Métodos
- **init(n_inputs, n_hidden, n_outputs)**: Inicializa las neuronas de la capa oculta y la capa de salida. Para la capa oculta se crean n_hidden neuronas cada una de las cuales tiene (n_inputs + 1) pesos, uno por cada columna en el conjunto de datos más el adicional por el peso $w_{0}$. Adicionalmente se crean n_outputs neuronas para la capa de salida, cada una con (n_hidden + 1) pesos. Esto significa que cada neurona en la capa de salida se conecta con cada neurona de la capa oculta.
- **forward_propagate(instance)**: Recibe en instance una instancia a clasificar, procesa la misma con cada neurona de la capa oculta, luego cada salida generada por las neuronas de la capa oculta son procesadas por cada una de las neuronas de la capa de salida, devolviendo finalmente el resultado de la clasificación.
- **update_weights(instance)**: Actualiza todos los pesos presentes en la red. Recibe la última instancia procesada como parámetro ya que son las inputs necesarias para actualizar pesos de la capa oculta.

In [11]:
class ANN:
    def __init__(self, n_inputs, n_hidden, n_outputs):
        self.hidden_layer = [Neuron(n_inputs + 1) for i in range(n_hidden)]
        self.output_layer = [Neuron(n_hidden + 1) for i in range(n_outputs)]

    def forward_propagate(self, instance):
        
        inputs = instance
        hidden_layer_outputs = []
        for neuron in self.hidden_layer:
            computed_input = neuron.compute_input(inputs)
            neuron_output = neuron.apply_sigmoid(computed_input)
            hidden_layer_outputs.append(neuron_output)
        
        inputs = hidden_layer_outputs
        final_outputs = []
        for neuron in self.output_layer:
            computed_input = neuron.compute_input(inputs)
            neuron_output = neuron.apply_sigmoid(computed_input)
            final_outputs.append(neuron_output)
        
        return final_outputs
    
    def update_weights(self, instance):
        
        output_layer_inputs = []
        for neuron in self.hidden_layer:
            neuron.update_weights(rate, instance)
            output_layer_inputs.append(neuron.output)
        
        for neuron in self.output_layer:
            neuron.update_weights(rate, output_layer_inputs)

### Parte a) <br>
### Algoritmo Backpropagation

In [39]:
def backpropagation(n_inputs, n_hidden, n_outputs, max_iter, data_set):
    
    # Crea la red neuronal artificial.     
    neural_network = ANN(n_inputs, n_hidden, n_outputs)
    
    for iter in range (max_iter):
        
        # Se recorre el conjunto de entrenamiento.         
        for (instance, expected) in data_set:
            
            # Se clasifica la instancia.             
            output = ANN.forward_propagate(instance)
            
            # Se recorren las neuronas de la capa de salida.             
            for index, neuron in enumerate(ANN.output_layer):
                # Se calcula y almacena el error para cada neurona.               
                neuron.calculate_output_neuron_error(expected[index])
                                     
            # Se recorren las neuronas de la capa oculta.
            for index, neuron in enumerate(ANN.hidden_layer):
                # Se calcula y almacena el error para cada neurona.               
                neuron.calculate_hidden_neuron_error(ANN.output_layer, index)

            # Se actualizan todos los pesos de la red.
            ANN.update_weights(instance)

### Parte b)

Se definen las funciones de la letra a utilizar:

In [47]:
def f(x):
    return x*x*x - x*x + 1

def g(x,y):
    return 1 - x*2 - y*2
    
def h(x,y):
    return x + y

[(x,f(x))for x in np.random.uniform(-1,1,40)]


[(-0.31805161914659008, 0.86667007321238798),
 (0.63734625121932464, 0.85268633224084001),
 (0.42814722694433538, 0.8951736407564731),
 (-0.30483303974794507, 0.87875076180873501),
 (0.039575142490400061, 0.99849579036459846),
 (0.35006468492449883, 0.92035349247208176),
 (-0.78947433874386519, -0.11532519069777347),
 (0.48634681109475419, 0.87850395692491678),
 (0.51094416881206572, 0.87232515604844685),
 (0.51497011976895735, 0.87137287717170375),
 (0.46684190569445461, 0.88380279721293931),
 (-0.65909583623306522, 0.27927662193603919),
 (0.88084179062240264, 0.90754725919782087),
 (-0.7338820852635568, 0.066160732129739608),
 (-0.23190101291515219, 0.93375072903274459),
 (0.1858114671486959, 0.9718894070556835),
 (0.84036984918460833, 0.88726575809282537),
 (0.16473894915655318, 0.97733191601839986),
 (-0.25301809527689945, 0.91978409143218054),
 (-0.50584392337687745, 0.61468755591097635),
 (-0.35718371690653372, 0.82685021961428318),
 (0.31022132531577262, 0.93361758297465713),
 (